# Module biogeme.database

## Examples of use of each function

This webpage is for programmers who need examples of use of the functions of the class. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2020-06-04 18:11:41.480594


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.6 [2020-06-04]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import pandas as pd
import numpy as np

In [4]:
import biogeme.database as db
from biogeme.expressions import Variable, exp, bioDraws

We set the seed so that the outcome of random operations is always the same.

In [5]:
np.random.seed(90267) 

## Create a database from a pandas data frame

In [6]:
df = pd.DataFrame({'Person':[1,1,1,2,2],
                   'Exclude':[0,0,1,0,1],
                   'Variable1':[1,2,3,4,5],
                   'Variable2':[10,20,30,40,50],
                   'Choice':[1,2,3,1,2],
                   'Av1':[0,1,1,1,1],
                   'Av2':[1,1,1,1,1],
                   'Av3':[0,1,1,1,1]})
myData = db.Database('test', df)
print(myData)

biogeme database test:
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3
0       1        0          1         10       1    0    1    0
1       1        0          2         20       2    1    1    1
2       1        1          3         30       3    1    1    1
3       2        0          4         40       1    1    1    1
4       2        1          5         50       2    1    1    1


## valuesFromDatabase

Evaluates an expression for each entry of the database.

        Args:
           expression: object of type biogeme.expressions. 

        Returns: 
           numpy series, long as the number of entries in the
           database, containing the calculated quantities.

In [7]:
Variable1 = Variable('Variable1')
Variable2 = Variable('Variable2')
expr = Variable1 + Variable2
result = myData.valuesFromDatabase(expr)
print(result)

0    11
1    22
2    33
3    44
4    55
dtype: int64


## checkAvailabilityOfChosenAlt

Check if the chosen alternative is available for each entry in the database.

        Args: 
            avail: list of biogeme.expressions to evaluate the
            availability conditions for each alternative.
            choice: biogeme.expressions to evaluate the chosen
            alternative.
                
        Returns:
           numpy series of bool, long as the number of entries 
           in the database, containing True is the chosen 
           alternative is available, False otherwise.


In [8]:
Av1 = Variable('Av1')
Av2 = Variable('Av2')
Av3 = Variable('Av3')
Choice = Variable('Choice')
avail = {1: Av1, 2: Av2, 3: Av3}
result = myData.checkAvailabilityOfChosenAlt(avail, Choice)
print(result)

0    False
1     True
2     True
3     True
4     True
dtype: bool


## sumFromDatabase

Calculates the value of an expression for each entry in the database, and retturns the sum.

        Args:
            expression: object of type biogeme.expressions 
            
        Returns:
            Sum of the expressions over the database.


In [9]:
Variable1 = Variable('Variable1')
Variable2 = Variable('Variable2')
expression = Variable2 / Variable1
result = myData.sumFromDatabase(expression)
print(result)

50.0


## Suggest scaling

Suggest a scaling of the variables in the database 
           
    Returns: 

        A Pandas dataframe where each row contains the name of
        the variable and the suggested scale s. Ideally, the column
        should be multiplied by s. If the suggested scaling is 1.0,
        10 or 0.1, it is not reported. 


In [10]:
myData.suggestScaling()

,Column,Scale,Largest
3,Variable2,0.01,50


## scaleColumn

Divide an entire column by a scale value

           Args:
              column: name of the column

              scale: value of the scale. All values of the 
              column will be multiplied by that scale.


In [11]:
myData.data

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3
0,1,0,1,10,1,0,1,0
1,1,0,2,20,2,1,1,1
2,1,1,3,30,3,1,1,1
3,2,0,4,40,1,1,1,1
4,2,1,5,50,2,1,1,1


In [12]:
myData.scaleColumn('Variable2', 0.01)

In [13]:
myData.data

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3
0,1,0,1,0.1,1,0,1,0
1,1,0,2,0.2,2,1,1,1
2,1,1,3,0.3,3,1,1,1
3,2,0,4,0.4,1,1,1,1
4,2,1,5,0.5,2,1,1,1


## addColumn

Add a new column in the database, calculated from an expression.

        Args:
           expression: object of type biogeme.expressions 
           describing the expression to evaluate
           column: name of the column to add.

        Returns:
           nothing

        Raises:
              ValueError: if the column name already exists.

In [14]:
Variable1 = Variable('Variable1')
Variable2 = Variable('Variable2')
expression = exp(0.5 * Variable2) / Variable1
expression = Variable2 * Variable1
result = myData.addColumn(expression, 'NewVariable')
print(myData.data['NewVariable'].tolist())

[0.1, 0.4, 0.8999999999999999, 1.6, 2.5]


In [15]:
myData.data

,Person,Exclude,Variable1,Variable2,Choice,Av1,Av2,Av3,NewVariable
0,1,0,1,0.1,1,0,1,0,0.1
1,1,0,2,0.2,2,1,1,1,0.4
2,1,1,3,0.3,3,1,1,1,0.9
3,2,0,4,0.4,1,1,1,1,1.6
4,2,1,5,0.5,2,1,1,1,2.5


## split

Shuffle the data, and split the data into slices. For each slide, an estimation and a validation sets are generated. The validation set is the slice itself. The estimation set is the rest of the data. 

In [17]:
dataSets = myData.split(3)
for i in dataSets:
    print("==========")
    print("Estimation:")
    print(type(i[0]))
    print(i[0])
    print("Validation:")
    print(i[1])

Estimation:
<class 'pandas.core.frame.DataFrame'>
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3  NewVariable
0       1        0          1        0.1       1    0    1    0          0.1
3       2        0          4        0.4       1    1    1    1          1.6
4       2        1          5        0.5       2    1    1    1          2.5
Validation:
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3  NewVariable
1       1        0          2        0.2       2    1    1    1          0.4
2       1        1          3        0.3       3    1    1    1          0.9
Estimation:
<class 'pandas.core.frame.DataFrame'>
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3  NewVariable
1       1        0          2        0.2       2    1    1    1          0.4
2       1        1          3        0.3       3    1    1    1          0.9
4       2        1          5        0.5       2    1    1    1          2.5
Validation:
   Person  Exclude  Variable1

## count

Counts the number of observations that have a specific value in a given column.

        Args:
            columnName: name of the column.
            value: value that is seeked.

        Returns: 
            Number of times that the value appears in the column.


Here, count the number of entries for individual 1.

In [ ]:
myData.count('Person',1)

## remove

Removes from the database all entries such that the value of the expression is not 0. 
        
        Args:
           expression: object of type biogeme.expressions 
           describing the expression to evaluate
        Returns:
           Nothing.

In [ ]:
Exclude = Variable('Exclude')
myData.remove(Exclude)
myData.data

## dumpOnFile

Dumps the database in a CSV formatted file.

        Returns:  name of the file

In [ ]:
myData.dumpOnFile()

In [ ]:
%%bash
cat test_dumped.dat

## generateDraws

Generate draws for each variable.
        
        Args:
             types:
                 A dict indexed by the names of the variables,
                 describing the types of draws. Each of them can be a
                 native type or any type defined by the function
                 database.setRandomNumberGenerators

             names: 
                 the list of names of the variables that require
                 draws to be generated.
             numberOfDraws: 
                 number of draws to generate.

        Returns: 
             a 3-dimensional table with draws. The 3 dimensions are
              1. number of individuals
              2. number of draws
              3. number of variables


List native types and their description

In [ ]:
myData.descriptionOfNativeDraws()

In [ ]:
randomDraws1 = bioDraws('randomDraws1', 'NORMAL_MLHS_ANTI')
randomDraws2 = bioDraws('randomDraws2', 'UNIFORM_MLHS_ANTI')
randomDraws3 = bioDraws('randomDraws3', 'UNIFORMSYM_MLHS_ANTI')

We build an expression that involves the three random variables

In [ ]:
x = randomDraws1 + randomDraws2 + randomDraws3
types = x.dictOfDraws()
print(types)

In [ ]:
theDrawsTable = myData.generateDraws(types,                         
                                     ['randomDraws1',
                                      'randomDraws2',
                                      'randomDraws3'],
                                     10)
theDrawsTable

## setRandomNumberGenerators

Defines user-defined random numbers generators.
        
        Args:

           rng: a dictionary of generators. The keys of the
           dictionary
           characterize the name of the generators, and must be
           different from the pre-defined generators in Biogeme:
           NORMAL, UNIFORM and UNIFORMSYM. The elements of the
           dictionary are functions that take two arguments: the
           number of series to generate (typically, the size of the
           database), and the number of draws per series.
 
        Returns: 
             nothing.

We first define functions returning draws, given the number of observations, and the number of draws

In [ ]:
def logNormalDraws(sampleSize, numberOfDraws):
    return np.exp(np.random.randn(sampleSize, numberOfDraws))

def exponentialDraws(sampleSize, numberOfDraws):
    return -1.0 * np.log(np.random.rand(sampleSize, numberOfDraws))

We associate these functions with a name

In [ ]:
dict = {'LOGNORMAL': (logNormalDraws, 
                      'Draws from lognormal distribution'), 
        'EXP': (exponentialDraws,
                'Draws from exponential distributions')}
myData.setRandomNumberGenerators(dict)

We can now generate draws from these distributions

In [ ]:
randomDraws1 = bioDraws('randomDraws1', 'LOGNORMAL')
randomDraws2 = bioDraws('randomDraws2', 'EXP')
x = randomDraws1 + randomDraws2
types = x.dictOfDraws()
theDrawsTable = myData.generateDraws(types,
                                     ['randomDraws1',
                                      'randomDraws2'],
                                     10)
print(theDrawsTable)

## sampleWithReplacement

Extract a random sample from the database, with replacement. Useful for bootstrapping. 
        Args:
            size: size of the sample. If None, a sample of the same size as the database will be generated.

        Returns:
            pandas dataframe with the sample.


In [ ]:
myData.sampleWithReplacement()

In [ ]:
myData.sampleWithReplacement(6)

## panel

Defines the data as panel data

        Args:
           columnName: name of the columns that identifies
           individuals.


In [ ]:
myPanelData = db.Database('test', df)

Data is not considered panel yet

In [ ]:
myPanelData.isPanel()

In [ ]:
myPanelData.panel('Person')

Now it is panel

In [ ]:
print(myPanelData.isPanel())

In [ ]:
print(myPanelData)

When draws are generated for panel data, a set of draws is generated per person, not per observation.

In [ ]:
randomDraws1 = bioDraws('randomDraws1', 'NORMAL')
randomDraws2 = bioDraws('randomDraws2', 'UNIFORM_HALTON3')

We build an expression that involves the two random variables

In [ ]:
x = randomDraws1 + randomDraws2
types = x.dictOfDraws()
theDrawsTable = myPanelData.generateDraws(types,
                                          ['randomDraws1',
                                           'randomDraws2'],
                                          10)
print(theDrawsTable)

## getNumberOfObservations

Reports the number of observations in the database. Note that it returns the same value, irrespectively if the database contains panel data or not.  

        Returns:
            Number of observations.

        See:  getSampleSize()


In [ ]:
myData.getNumberOfObservations()

In [ ]:
myPanelData.getNumberOfObservations()

## getSampleSize

Reports the size of the sample. If the data is cross-sectional, it
        is the number of observations in the database. If the data is panel,
        it is the number of individuals.

        Returns: 
           Sample size.

        See: getNumberOfObservations()


In [ ]:
myData.getSampleSize()

In [ ]:
myPanelData.getSampleSize()

## sampleIndividualMapWithReplacement

Extract a random sample of the individual map from a panel data database, with replacement. Useful for bootstrapping. 

        Args:
            size: size of the sample. If None, a sample of the same
            size as the database will be generated.

        Returns:
            pandas dataframe with the sample.


In [ ]:
myPanelData.sampleIndividualMapWithReplacement(10)

## sampleWithoutReplacement

It is possible as well to sample without replacement. This is typically useful for stochastic algorithms that use only part of the database. 

In [ ]:
df = pd.DataFrame({'Person': [1, 1, 1, 2, 2],
                   'Exclude': [0, 0, 1, 0, 1],
                   'Variable1': [1, 2, 3, 4, 5],
                   'Variable2': [10, 20, 30, 40, 50],
                   'Choice': [1, 2, 3, 1, 2],
                   'Av1': [0, 1, 1, 1, 1],
                   'Av2': [1, 1, 1, 1, 1],
                   'Av3': [0, 1, 1, 1, 1],
                   'Weight': [1, 5, 1, 1, 5]})
myData = db.Database('test', df)
myData.data

In [ ]:
myData.data.Choice.value_counts()

In [ ]:
myData.sampleWithoutReplacement(0.7)
myData.data

In [ ]:
myData.data.Choice.value_counts()

The sampling does not have to be uniform. Here, we oversample data corresponding to Choice = 2

In [ ]:
myData.sampleWithoutReplacement(0.7, 'Weight')
myData.data

In [ ]:
myData.data.Choice.value_counts()